In [9]:
# Bibliothèques
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from mpl_toolkits.mplot3d import Axes3D

# Data loading
data = pd.read_csv('/Users/debidour/Desktop/ENSC/PERSONNEL/3A/Landmining_project_ENSC/Mine_Dataset.csv', header=None)

# Features V, H, S, classe (M)
feature_names = ['V', 'H', 'S']
X = data.iloc[:, :-1].values  # Les features
y = data.iloc[:, -1].values  # La variable cible (M)

# Étape 1 : format variable unique dans y 
print("Valeurs uniques dans y : ", np.unique(y))

# Étape 2 : Remplacer les chaînes de caractères par des valeurs numériques
y_cleaned = np.where(y == 'M', 0, y)

# Conversion finale des valeurs en entiers après nettoyage
y_cleaned = y_cleaned.astype(int)

# Vérification après nettoyage
print("Valeurs uniques dans y_cleaned : ", np.unique(y_cleaned))

y_cleaned = y_cleaned - 1

# 1. Visualisation 2D avec seaborn : Paires de variables --> DataFrame
df = pd.DataFrame(X, columns=feature_names)
df['Class'] = y_cleaned

# Utilisation de pairplot pour voir les paires de variables avec les classes colorées
sns.pairplot(df, hue='Class', palette="Set1", markers=["o", "s", "D", "v", "^"])
plt.suptitle("Visualisation des paires de variables", y=1.02)
plt.show()

# 2. Visualisation 3D pour toutes les features
fig = plt.figure(figsize=(10, 8))
ax = fig.add_subplot(111, projection='3d')

# Création d'un scatter plot 3D
sc = ax.scatter(X[:, 0], X[:, 1], X[:, 2], c=y_cleaned, cmap='coolwarm', s=50)
ax.set_xlabel('V')
ax.set_ylabel('H')
ax.set_zlabel('S')

plt.title("Visualisation 3D des features")
plt.colorbar(sc, label='Classes')
plt.show()


Valeurs uniques dans y :  ['1' '2' '3' '4' '5' 'M']
Valeurs uniques dans y_cleaned :  [0 1 2 3 4 5]


ValueError: No variables found for grid columns.

Test suivant à réaliser : 
- Tout d'abord, séparer les données de test des données de training ;
- augmenter avec llm les données de train (augmentation)
- cleanlab python pour nettoyer les données 
- trainer le modèle
- tester le modèle avec les datas normales 

In [2]:
# Librairies
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, confusion_matrix, roc_curve, auc
import matplotlib.pyplot as plt
import seaborn as sns
from cleanlab.classification import CleanLearning
from sklearn.ensemble import RandomForestClassifier  # Optionnel pour comparaison

# Dataset

data = pd.read_csv('/Users/debidour/Desktop/ENSC/PERSONNEL/3A/Landmining_Project_ENSC/Mine_Dataset.csv')

# Prétraitement données test - train

data.fillna(0, inplace=True)

# Étape 1 : Séparation des données de test

X = data.drop(columns=['M'])  # 'M' est la variable cible : le type de mine 
y = data['M']
X_train_full, X_test, y_train_full, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Étape 2 : Amélioration des données d'entraînement avec un LLM (augmentation synthétique)

# Exemple : Dupliquer les données bruyées pour simuler une augmentation : à voir usage d'un LLM simple pour ce faire à la place

X_augmented = X_train_full + np.random.normal(0, 0.01, X_train_full.shape)
y_augmented = y_train_full.copy()
X_train_full = pd.concat([X_train_full, pd.DataFrame(X_augmented, columns=X_train_full.columns)])
y_train_full = pd.concat([y_train_full, y_augmented])

# Étape 3 : Nettoyage des données avec Cleanlab

base_model = RandomForestClassifier(random_state=42)
cleanlab_model = CleanLearning(clf=base_model)
cleanlab_model.fit(X_train_full, y_train_full)
cleaned_indices = cleanlab_model.get_noise_indices()

# Supprimer les données identifiées comme bruitées

X_train_cleaned = X_train_full.iloc[~cleaned_indices]
y_train_cleaned = y_train_full.iloc[~cleaned_indices]

# Étape 4 : Standardisation des données nettoyées

scaler = StandardScaler()
X_train_scaled = scaler.fit_transform(X_train_cleaned)
X_test_scaled = scaler.transform(X_test)

# Étape 5 : Entraînement du modèle

model = LogisticRegression()
model.fit(X_train_scaled, y_train_cleaned)

# Étape 6 : Test sur les données normales

y_pred = model.predict(X_test_scaled)
y_prob = model.predict_proba(X_test_scaled)[:, 1]

# Rapport de classification

print("Rapport de classification :")
print(classification_report(y_test, y_pred))

# Matrice de confusion

conf_matrix = confusion_matrix(y_test, y_pred)
plt.figure(figsize=(6, 4))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap='Blues')
plt.title('Matrice de confusion')
plt.ylabel('Valeurs réelles')
plt.xlabel('Prédictions')
plt.show()

# Courbe ROC
fpr, tpr, _ = roc_curve(y_test, y_prob)
roc_auc = auc(fpr, tpr)

plt.figure(figsize=(8, 6))
plt.plot(fpr, tpr, label=f'AUC = {roc_auc:.2f}', color='darkorange')
plt.plot([0, 1], [0, 1], color='navy', linestyle='--')
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Taux de faux-positifs')
plt.ylabel('Taux de vrai-positifs')
plt.title('Receiver Operating Characteristic (Courbe ROC)')
plt.legend(loc="lower right")
plt.show()

ValueError: pred_probs must have at least 6 columns, based on the largest class index which appears in labels.